<a href="https://colab.research.google.com/github/FaiqaIftikhar/.well-known/blob/master/resNetSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2

In [ ]:
#emotion_data = pd.read_csv('/content/drive/My Drive/fer2013.csv')
#print(emotion_data)
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link='1E6PXNr_PBkvOImxb_YVR88wGm3SMRNs-'
#fluff, id = link.split('=')
#print (id)
downloaded = drive.CreateFile({'id':link}) 
downloaded.GetContentFile('fer2013.csv')  
emotion_data = pd.read_csv('fer2013.csv')
print(emotion_data)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []
for index, row in emotion_data.iterrows():
    if row['Usage'] == 'Training':
      k = row['pixels'].split(" ")
      X_train.append(np.array(k,'float32'))
      y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
      k = row['pixels'].split(" ")
      X_test.append(np.array(k,'float32'))
      y_test.append(row['emotion'])
  

print(k)

['61', '63', '59', '75', '151', '159', '166', '161', '143', '170', '127', '131', '184', '216', '222', '52', '70', '89', '83', '97', '93', '92', '88', '85', '89', '83', '85', '76', '67', '60', '60', '57', '31', '90', '136', '100', '122', '111', '71', '50', '92', '86', '88', '94', '82', '88', '81', '87', '60', '63', '54', '92', '158', '161', '162', '165', '138', '185', '199', '200', '212', '239', '81', '37', '92', '103', '100', '101', '104', '102', '99', '100', '101', '98', '97', '95', '91', '75', '63', '58', '54', '50', '156', '142', '119', '130', '84', '63', '74', '80', '82', '94', '92', '85', '83', '84', '58', '65', '53', '118', '161', '169', '154', '162', '163', '173', '197', '229', '210', '59', '33', '79', '98', '107', '108', '103', '104', '108', '109', '110', '113', '115', '118', '114', '109', '100', '85', '65', '61', '38', '118', '190', '132', '123', '104', '84', '56', '70', '78', '83', '89', '80', '91', '81', '59', '64', '57', '132', '153', '163', '175', '159', '192', '182', '197

In [ ]:
from keras.utils import np_utils

num_features = 64
num_labels = 7
batch_size = 32
epochs = 5
width, height = 48, 48


X_train = np.array(X_train,'float32')
y_train = np.array(y_train,'float32')
X_test = np.array(X_test,'float32')
y_test = np.array(y_test,'float32')

y_train= np_utils.to_categorical(y_train, num_classes=7)
y_test = np_utils.to_categorical(y_test, num_classes=7)

#cannot produce
#normalizing data between oand 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [ ]:
stride = 1
CHANNEL_AXIS = 3

def res_layer(x ,filters,pooling = False,dropout = 0.0):
    temp = x
    temp = Conv2D(filters,(3,3),strides = stride,padding = "same")(temp)
    temp = BatchNormalization(axis = CHANNEL_AXIS)(temp)
    temp = Activation("relu")(temp)
    temp = Conv2D(filters,(3,3),strides = stride,padding = "same")(temp)

    x = add([temp,Conv2D(filters,(3,3),strides = stride,padding = "same")(x)])
    if pooling:
        x = MaxPooling2D((2,2))(x)
    if dropout != 0.0:
        x = Dropout(dropout)(x)
    x = BatchNormalization(axis = CHANNEL_AXIS)(x)
    x = Activation("relu")(x)
    return x


In [ ]:
from keras.layers import BatchNormalization
from keras.layers import add
from keras.layers import Input,Activation
from keras.models import Model


kernel_size = (3, 3)
input_shape=X_train.shape[1:]
inp = Input(shape=input_shape)
x = inp
x = Conv2D(16,(3,3),strides = stride,padding = "same")(x)
x = BatchNormalization(axis = CHANNEL_AXIS)(x)
x = Activation("relu")(x)
x = res_layer(x,32,dropout = 0.2)
x = res_layer(x,32,dropout = 0.3)
x = res_layer(x,32,dropout = 0.4,pooling = True)
x = res_layer(x,64,dropout = 0.2)
x = res_layer(x,64,dropout = 0.2,pooling = True)
x = res_layer(x,256,dropout = 0.4)
x = Flatten()(x)
x = Dropout(0.4)(x)
x = Dense(4096,activation = "relu")(x)
x = Dropout(0.23)(x)
x = Dense(7,activation = "softmax")(x)

resnet_model = Model(inp,x,name = "Resnet")
resnet_model.summary()

Model: "Resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 48, 48, 16)   160         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 48, 48, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 48, 48, 16)   0           batch_normalization[0][0]        
_____________________________________________________________________________________________

In [ ]:
resnet_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
resnet_model.fit(X_train,y_train,batch_size=32,epochs=30,verbose=1,validation_data=(X_test, y_test))

Epoch 1/30
898/898 [==============================] - 67s 66ms/step - loss: 6.2667 - accuracy: 0.2411 - val_loss: 1.7550 - val_accuracy: 0.2940
Epoch 2/30
898/898 [==============================] - 59s 65ms/step - loss: 1.7537 - accuracy: 0.2835 - val_loss: 1.6283 - val_accuracy: 0.3558
Epoch 3/30
898/898 [==============================] - 59s 66ms/step - loss: 1.6237 - accuracy: 0.3536 - val_loss: 1.4810 - val_accuracy: 0.4160
Epoch 4/30
898/898 [==============================] - 59s 66ms/step - loss: 1.5284 - accuracy: 0.3985 - val_loss: 1.5085 - val_accuracy: 0.4146
Epoch 5/30
898/898 [==============================] - 60s 67ms/step - loss: 1.4619 - accuracy: 0.4340 - val_loss: 1.3730 - val_accuracy: 0.4748
Epoch 6/30
898/898 [==============================] - 60s 67ms/step - loss: 1.4083 - accuracy: 0.4507 - val_loss: 1.4018 - val_accuracy: 0.4717
Epoch 7/30
898/898 [==============================] - 60s 67ms/step - loss: 1.3560 - accuracy: 0.4768 - val_loss: 1.3041 - val_accuracy:

In [ ]:
# predict probabilities for test set
yhat_probs = resnet_model.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = resnet_model.predict(X_test, verbose=0)
# reduce to 1d array
#yhat_probs = yhat_probs[:, 0]
#yhat_classes = yhat_classes[:, 0]
#y_test = y_test[:, 0]
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]
print(y_test)
print(yhat_classes)

[1. 0. 0. ... 0. 0. 0.]
[0.25846094 0.02656565 0.6691495  ... 0.00263612 0.79040784 0.15040687]


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes.round())
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes.round(),pos_label='positive',average='macro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes.round(),pos_label='positive',average='macro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes.round(),pos_label='positive',average='macro')
print('F1 score: %f' % f1)

Accuracy: 0.886598
Precision: 0.750574
Recall: 0.715384
F1 score: 0.730892


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
